In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import warnings
warnings.filterwarnings('always')
from sklearn.model_selection import train_test_split
import shapely.speedups
from sklearn.preprocessing import StandardScaler
from IPython.display import Image
import re
from itertools import combinations
import random
import joblib

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.preprocessing import StandardScaler

import tensorflow as tf
from tensorflow import keras
import keras_tuner as kt
#
np.random.seed(1)
tf.random.set_seed(1)

In [ ]:
def plot_regression(modelo,x,y,title="",xlabel="x",ylabel="y"):
    plt.figure()
    
    plt.plot(x,y,"o",label="Valores verdaderos")
    plt.plot(x,modelo.predict(x),"x",label="Valores estimados")
    
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.title(title)
    plt.legend()
    plt.show()

In [ ]:
viviendas_train=pd.read_csv('./TP1/Datasets/viviendas_caba_train.csv').copy()
viviendas_test=pd.read_csv('./TP1/Datasets/viviendas_caba_test.csv').copy()
viviendas = pd.concat([viviendas_train, viviendas_test])

In [ ]:
viviendas = pd.get_dummies(viviendas, columns = ["property_type"])
viviendas = pd.get_dummies(viviendas, columns = ["barrio"])

In [ ]:
columnas_predictoras=viviendas.columns.to_list()
for variable in ['property_price', 'start_date', 'end_date', 'property_title', 'geometry']:
    columnas_predictoras.remove(variable)
columnas_predictoras

In [ ]:
variable_predictora="property_price"

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(viviendas[columnas_predictoras],viviendas[[variable_predictora]],test_size=0.2)

In [ ]:
sscaler=StandardScaler()
sscaler.fit(pd.DataFrame(x_train))

In [ ]:
x_train_transform=sscaler.transform(pd.DataFrame(x_train))
x_test_transform=sscaler.transform(pd.DataFrame(x_test))

In [ ]:
# Creo un modelo Red Neuronal
d_in=x_test_transform.shape[1]
d_out=1

modelo3 = keras.Sequential([
    # input_shape solo en la primer capa
    # Capa con 2 salidas, activación relu
    keras.layers.Dense(2,input_shape=(d_in,), activation="relu"),
    # Capa con 2 salidas, activación tanh
    keras.layers.Dense(50, activation="relu" ),
    keras.layers.Dense(4, activation="relu" ),
    keras.layers.Dense(6, activation="relu" ),
    keras.layers.Dense(8, activation="relu" ),



    keras.layers.Dense(d_out, )])

modelo3.compile(
  optimizer=keras.optimizers.Adadelta(learning_rate=0.1), 
  loss='mse', 
  # metricas para ir calculando en cada iteracion o batch 
  metrics=['mse'], 
)

#modelo3.fit(x_train_transform,y_train,epochs=100,batch_size=16,verbose=False)


In [ ]:
y_pred_modelo3=modelo3.predict(x_test_transform)

mae_modelo3=mean_absolute_error(y_test,y_pred_modelo3)
mse_modelo3=mean_squared_error(y_test,y_pred_modelo3)

print(f"Error absoluto medio {mae_modelo3}")
print(f"Error cuadrático medio {mse_modelo3}")
print(f"rmse {mse_modelo3**(1/2)}")

In [ ]:
y_pred_modelo3

In [ ]:
scatter_real_predict = plt.scatter(x=y_test, y=y_pred_modelo3, s=10, c='tab:blue')#.set(title="Valor Real vs Predicción",xlabel='Valor Real',ylabel='Prediccion') 

## RANDOMIZADOR


In [ ]:
class ModeloRedes:
    def __init__(self, dataset, variables_predictoras, cantidad_de_primera_capa, funcion_activacion, optimizador, learning_rate, metrica, epoch, batch_size, cantidad_filas):
        self.variables_predictoras = variables_predictoras
        self.cantidad_primera_capa = cantidad_de_primera_capa
        self.funcion_activacion = funcion_activacion
        self.optimizador = optimizador
        self.learning_rate = learning_rate
        self.resultado = None
        self.metrica = metrica
        self.epoch = epoch
        self.batch_size = batch_size
        self.modelo = None

        
        self.x_test = None
        self.y_test = None
        self.x_train = None
        self.y_train = None
        self.dataset_trabajo = dataset[:cantidad_filas]
        self.obtener_datasets()
        self.entrenar_modelo()

    #HIPERPARAMETROS: /cantidad de predictoras , cantidad de layer, neuronas por leyers, optimizador, learning rate, funcion de activacion

    def obtener_datasets(self):    
        self.x_train, self.x_test, self.y_train, self.y_test = train_test_split(self.dataset_trabajo.loc[:,self.variables_predictoras],self.dataset_trabajo.loc[:,'property_price'],test_size=0.2)
        self.escalar_datasets()
    
    def escalar_datasets(self):
        sscaler=StandardScaler()
        sscaler.fit(pd.DataFrame(self.x_train))
        self.x_train=sscaler.transform(pd.DataFrame(self.x_train))
        self.x_test=sscaler.transform(pd.DataFrame(self.x_test))

    def entrenar_modelo(self):
    # Creo un modelo Red Neuronal
        d_in= len(self.variables_predictoras)
        d_out=1

        capas = [
            # input_shape solo en la primer capa
            # Capa con 2 salidas, activación relu
            keras.layers.Dense(d_in,input_shape=(d_in,), activation= self.funcion_activacion)
        ]

        #inserta capas que van decreciendo en cantidad de neuronas
        for i in range(self.cantidad_primera_capa,2,-1):
            capas.append(keras.layers.Dense(i, activation= self.funcion_activacion ))

        capas.append(keras.layers.Dense(d_out, ))
        self.modelo = keras.Sequential(capas)

        self.modelo.compile(
        optimizer= self.optimizador(self.learning_rate), 
        loss=self.metrica, 
        # metricas para ir calculando en cada iteracion o batch 
        metrics=[self.metrica], 
        )

        self.modelo.fit(self.x_train,self.y_train,epochs=self.epoch,batch_size=self.batch_size,verbose=False)
    
    def imprimir(self):
        plot_regression(self.modelo,self.x_test,self.y_test,title=f"Modelo Final",xlabel="Horsepower",ylabel="MPG")
        
    def testear_modelo(self):
        y_pred=self.modelo.predict(self.x_test)
        mse=mean_squared_error(self.y_test,y_pred)
        self.resultado = mse ** (1/2)
        return self.resultado
    
    def predecir(self, df):

        df_para_predecir =  df.loc[:,self.variables_predictoras]
        df_precios = df.loc[:,'property_price']
        
        sscaler=StandardScaler()
        sscaler.fit(pd.DataFrame(df_para_predecir))
        df_para_predecir=sscaler.transform(pd.DataFrame(df_para_predecir))

        
        precios_pred=self.modelo.predict(df_para_predecir)
        mse=mean_squared_error(df_precios,precios_pred)
        resultado = mse ** (1/2)
        #plot_regression(self.modelo,df_para_predecir,df_precios,title=f"Modelo Final",xlabel="Horsepower",ylabel="MPG")
        return resultado
    
    def info(self):
        print('\n','\n')
        print('\n',
        self.variables_predictoras , '\n',
        self.cantidad_primera_capa ,'\n',
        self.funcion_activacion ,'\n',
        self.optimizador ,'\n',
        self.learning_rate ,'\n',
        self.metrica ,'\n',
        self.epoch ,'\n',
        self.batch_size ,'\n',
        self.dataset_trabajo.shape ,'\n')

In [ ]:
columnas_importantes =  ['latitud', 'longitud', 'property_surface_total', 'property_surface_covered']
columnas_extras = []
for e in columnas_predictoras:
    if e not in columnas_importantes:
        columnas_extras.append(e)

print(columnas_extras)        

def agregar_columnas(posibles_variables_predictoras, columnas, cant_columnas):
            i = 0
            while (i < cant_columnas): 
                pos = random.randrange(i,len(columnas), 1)
                posibles_variables_predictoras.append(columnas[pos])
                columnas[i], columnas[pos] = columnas[pos], columnas[i]
                i += 1

posibles_cantidad_de_primera_capa = []
for i in range(1,20):
    posibles_cantidad_de_primera_capa.append(i)

posibles_funcion_activacion = ["relu"]#, "tanh", "softmax", "sigmoid"]
posibles_metricas = ['mse', 'mae']

posibles_learning_rate = [0.1, 0.01]

In [ ]:
def funcion_optimizadoraNADAM(learnigRate):
            return keras.optimizers.Nadam(learning_rate = learnigRate)

def funcion_optimizadoraRMSPROP(learnigRate):
            return keras.optimizers.RMSprop(learning_rate = learnigRate)

def funcion_optimizadoraADAMAX(learnigRate):
            return keras.optimizers.Adamax(learning_rate = learnigRate)

def funcion_optimizadoraADADELTA(learnigRate):
            return keras.optimizers.Adadelta(learning_rate = learnigRate)

posibles_optimizador = [funcion_optimizadoraNADAM, funcion_optimizadoraRMSPROP, funcion_optimizadoraADAMAX, funcion_optimizadoraADADELTA]

In [ ]:
def random_validator(cantidad_modelos, columnas_importantes, columnas_extras, posibles_cantidad_de_primera_capa, 
    posibles_funcion_activacion, posibles_optimizador, posibles_learning_rate, posibles_metricas):
    resultados = []
    for i in range(cantidad_modelos):

        posibles_variables_predictoras = []
        cant_columnas_importantes = random.randrange(1,len(columnas_importantes)+1, 1)
        cant_columnas_extras = random.randrange(0,len(columnas_extras)+1, 1)

        agregar_columnas(posibles_variables_predictoras, columnas_importantes, cant_columnas_importantes)
        agregar_columnas(posibles_variables_predictoras, columnas_extras, cant_columnas_extras)

        variables_predictoras = posibles_variables_predictoras
        cantidad_de_primera_capa = random.choice(posibles_cantidad_de_primera_capa)
        funcion_activacion = random.choice(posibles_funcion_activacion)
        optimizador = random.choice(posibles_optimizador)
        learning_rate = random.choice(posibles_learning_rate)
        metrica = random.choice(posibles_metricas)
        epoch = random.randrange(100, 300, 10)
        batch_size = random.randrange(10, 1000, 1)
        cantidad_filas = random.randrange(3000, 5000, 1)

        print('\n','\n')
        print(i , '\n',
        variables_predictoras , '\n',
        cantidad_de_primera_capa ,'\n',
        funcion_activacion ,'\n',
        optimizador ,'\n',
        learning_rate ,'\n',
        metrica ,'\n',
        epoch ,'\n',
        batch_size ,'\n',
        cantidad_filas ,'\n')

        modelo = ModeloRedes(viviendas, variables_predictoras, cantidad_de_primera_capa, funcion_activacion, optimizador, learning_rate, metrica, epoch, batch_size, cantidad_filas)
        resultados.append([modelo, modelo.testear_modelo()])
        print(modelo.testear_modelo())
    return resultados

In [ ]:
def func():
    resultados = random_validator(50,columnas_importantes, columnas_extras, posibles_cantidad_de_primera_capa, 
        posibles_funcion_activacion, posibles_optimizador, posibles_learning_rate, posibles_metricas)

    return resultados

In [ ]:
def reentrenar(modelo):
    modelo.info()
    modelo.testear_modelo()
    mejor_modelo = modelo
    mejor_resultado = modelo.resultado 
   
    for i in range(5):
        print('valor a vencer', mejor_resultado)
        modelo_aux = ModeloRedes(viviendas, modelo.variables_predictoras, modelo.cantidad_primera_capa, modelo.funcion_activacion, modelo.optimizador, modelo.learning_rate, modelo.metrica, modelo.epoch, modelo.batch_size, 3000)
        resultado_aux = modelo_aux.testear_modelo()
        print('reentrenando intento', i+1)
        resultado_aux = modelo_aux.testear_modelo()
        print(resultado_aux)
        modelo.info()
        
        if resultado_aux < mejor_resultado:
            mejor_resultado = resultado_aux
            mejor_modelo = modelo_aux 
    return [mejor_modelo, mejor_resultado]

In [ ]:
def obtener_mejores(resultados):
    mejores = []
    for i in resultados:
        if (i[1] < 130000):
            mejores.append(i)

    for i in range(len(mejores)):
        mejores[i] = reentrenar(mejores[i][0])
    return mejores

In [ ]:
mejores = []

In [ ]:

resultados = func()
mejores.append(obtener_mejores(resultados))

In [ ]:

resultados = func()
mejores.append(obtener_mejores(resultados))

In [ ]:

resultados = func()
mejores.append(obtener_mejores(resultados))

In [85]:
for i in mejores:
    for j in i:
        print(j)
        #j[0].info()

[<__main__.ModeloRedes object at 0x7f4123f36550>, 119333.58843831343, 134943.4504846082, 134943.4504846082, 134943.4504846082]
[<__main__.ModeloRedes object at 0x7f4109d6cbb0>, 102451.9140242622, 155414.72864861964, 155414.72864861964, 155414.72864861964]
[<__main__.ModeloRedes object at 0x7f415f4fe610>, 122550.37412418632, 134653.48012445797, 134653.48012445797, 134653.48012445797]
[<__main__.ModeloRedes object at 0x7f41881be6d0>, 103213.98013599498, 138393.87250814715, 138393.87250814715]
[<__main__.ModeloRedes object at 0x7f412732bbe0>, 109137.77611246107, 145345.2228765837, 145345.2228765837]
[<__main__.ModeloRedes object at 0x7f4124554580>, 117347.09238797102, 162116.938124902, 162116.938124902]
[<__main__.ModeloRedes object at 0x7f41485572b0>, 96759.78693229835, 142374.07943027507, 142374.07943027507]
[<__main__.ModeloRedes object at 0x7f4108ba5a60>, 114676.93486053028, 154137.29854784868, 154137.29854784868]
[<__main__.ModeloRedes object at 0x7f41166398b0>, 110141.21347729258, 1

In [ ]:
for i in mejores:
    for j in i:
        print('El modelo tiene metrica')
        print(j[1])
        j[0].info()
        j[0].imprimir()
        print('El modelo predice sobre todo el dataset como')
        j.append(j[0].predecir(viviendas))
        print(j[2])


In [ ]:
"""va = ['longitud', 'property_surface_total', 'property_surface_covered', 'latitud', 'barrio_Paternal', 'barrio_Retiro', 'barrio_Las Cañitas', 'barrio_Pompeya', 'barrio_Balvanera', 'barrio_Monserrat', 'barrio_Villa Pueyrredón', 'barrio_Mataderos', 'barrio_Chacarita', 'barrio_Barracas', 'property_bedrooms', 'barrio_San Nicolás', 'barrio_Villa General Mitre', 'barrio_Liniers', 'barrio_Villa Santa Rita', 'barrio_San Telmo', 'property_type_Departamento', 'barrio_Villa Lugano', 'barrio_Almagro', 'barrio_Coghlan', 'barrio_Once', 'barrio_Villa Soldati', 'barrio_Villa Ortuzar', 'barrio_Floresta', 'barrio_Villa Riachuelo', 'barrio_Constitución', 'barrio_Congreso', 'barrio_Velez Sarsfield', 'barrio_Villa del Parque', 'barrio_Villa Devoto', 'barrio_Abasto', 'barrio_Villa Urquiza', 'barrio_Catalinas', 'barrio_Recoleta', 'barrio_Caballito', 'barrio_Nuñez', 'barrio_Barrio Norte', 'barrio_Versalles', 'barrio_Monte Castro', 'barrio_Boedo', 'barrio_Belgrano', 'barrio_Boca', 'barrio_Parque Avellaneda', 'barrio_Tribunales', 'barrio_Villa Real', 'barrio_Villa Crespo', 'barrio_Centro / Microcentro', 'property_rooms', 'barrio_Parque Patricios', 'property_type_PH', 'barrio_Agronomía', 'barrio_Colegiales', 'barrio_Flores', 'barrio_Saavedra', 'barrio_San Cristobal', 'property_type_Casa', 'barrio_Puerto Madero', 'barrio_Parque Centenario'] 

modelo = ModeloRedes(viviendas, va, 15, 'relu', funcion_optimizadoraADADELTA, 0.01, 'mae', 200, 125, 2000)
print(modelo.testear_modelo())
modelo.info()
modelo.imprimir()"""

In [ ]:
"""modelo = mejores[0][0][0]

df_para_predecir =  viviendas.loc[:,modelo.variables_predictoras]
df_precios = viviendas.loc[:,'property_price']

sscaler=StandardScaler()
sscaler.fit(pd.DataFrame(df_para_predecir))
df_para_predecir=sscaler.transform(pd.DataFrame(df_para_predecir))


precios_pred=modelo.modelo.predict(df_para_predecir)
mse=mean_squared_error(df_precios,precios_pred)
print (mse ** (1/2))
plot_regression(modelo.modelo,df_para_predecir,df_precios,title=f"Modelo Final",xlabel="Horsepower",ylabel="MPG")"""

In [ ]:
#joblib.dump(mejores[0][0][0], './Models/red_regresion_67k') 